# Homework 7 (30 marks)
Create a copy of the notebook to start answering the questions. Name your notebook in the format HW7_lastname_firstname.ipynb to facilitate the grading process.

Answer all the questions, test your code to ensure there are no errors and the results are as expected. Once you have answered all the questions, save the final copy, then go to File-> click on Download.ipynb. Once the local copy has been downloaded, submit your file on Blackboard under the corresponding assignment section. Also provide us a link to your notebook during submission.

NOTE: Please give the TAs the permission to access your notebooks through the links you have provided during submission.

The due date of this homework is 04/23/2021 (Friday).

Please ensure you follow all the steps mentioned in the homework.

You can submit your solutions any number of times until the deadline.

The datasets used in this homework can be found in the google drive link below -

https://drive.google.com/drive/folders/1NxCh4X7u7wVo5aHojxjLNs9wC7B7zJhb?usp=sharing

Follow the necessary steps to import data to test your code. You can use any method to read your data in the notebook. We will not be grading the methods you use. We will only grade the code from the point where you read the dataset into a pandas dataframe - (pd.read_csv('file_name'))

Import all the libraries you require in the cell below.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Enter your code here

## Implement the Ensemble methods learnt in class and compare their accuarcies.

The dataset you are going to be using for homework is the **Wisconsin Breast Cancer dataset (cancer.csv)**

The dataset contains a total number of 10 features labeled in either benign or malignant classes. The features have 699 instances out of which 16 feature values are missing. The dataset only contains numeric values.

Attribute Information:

1. Sample code number: id number
2. Clump Thickness: 1 - 10
3. Uniformity of Cell Size: 1 - 10
4. Uniformity of Cell Shape: 1 - 10
5. Marginal Adhesion: 1 - 10
6. Single Epithelial Cell Size: 1 - 10
7. Bare Nuclei: 1 - 10
8. Bland Chromatin: 1 - 10
9. Normal Nucleoli: 1 - 10
10. Mitoses: 1 - 10
11. Class: (2 for benign, 4 for malignant) (**target variable**)

For more information: https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)

### 1. Read the dataset into variable called '**data**' (1 mark)

In [ ]:
pd.set_option('display.max_columns', 100)
# Enter your code here
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_handle = drive.CreateFile({'id': '17vncPldmayk3pnyO5kuSquNrkJ8zJyL_'}) 
file_handle.GetContentFile('cancer.csv')
data = pd.read_csv('cancer.csv')


### **Preprocessing**: Data needs to be preprocessed before implementing ensemble methods. It is done for you here. 
### Run the below code first and then answer the questions from 2 - 7.

#### Deleting unnecessary columns: The column "Sample code number" is just an indicator and it's of no use in the modeling. So, let's drop it:


In [ ]:
data.drop(['Sample Code Number'],axis = 1, inplace = True)


#### Handling missing values : 
As mentioned earlier, the dataset contains missing values. The column named "Bare Nuclei" contains them. The missing values are represneted as "?". 

Replace those "?"s with 0's and impute them with Mean Imputation

In [ ]:
data['Bare Nuclei']

0       3
1       3
2       3
3       3
4       3
       ..
694     1
695     1
696     8
697    10
698    10
Name: Bare Nuclei, Length: 699, dtype: int64

In [ ]:
data.replace('?',0, inplace=True)

In [ ]:
# Convert the DataFrame object into NumPy array otherwise you will not be able to impute
values = data.values
# Now impute it
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputedData = imputer.fit_transform(values)

#### Normalizing the data:
Ranges of the features of the dataset are not the same. This may cause a problem. A small change in a feature might not affect the other. To address this problem, normalize the ranges of the features to a uniform range, in this case, 0 - 1.

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
normalizedData = scaler.fit_transform(imputedData)
cols = ['Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bland Chromatin', 'Bare Nuclei', 'Normal Nucleoli', 'Mitosis','Class']
normalizedData = pd.DataFrame(normalizedData, columns=cols)
print(normalizedData.head())

   Clump Thickness  Uniformity of Cell Size  Uniformity of Cell Shape  \
0         0.444444                 0.000000                  0.000000   
1         0.444444                 0.333333                  0.333333   
2         0.222222                 0.000000                  0.000000   
3         0.555556                 0.777778                  0.777778   
4         0.333333                 0.000000                  0.000000   

   Marginal Adhesion  Single Epithelial Cell Size  Bland Chromatin  \
0           0.000000                     0.111111              0.1   
1           0.444444                     0.666667              1.0   
2           0.000000                     0.111111              0.2   
3           0.000000                     0.222222              0.4   
4           0.222222                     0.111111              0.1   

   Bare Nuclei  Normal Nucleoli  Mitosis  Class  
0     0.222222         0.000000      0.0    0.0  
1     0.222222         0.111111      0.0

### Data preprocessing is done and now you will answer the below questions using the **normalizedData**: 

### 2. Split the data into test and training data with test size - 30%. Compute the baseline classification accuracy for X_train. (3 marks)

In [ ]:
# Enter your code here
feature_cols = ['Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bland Chromatin', 'Bare Nuclei', 'Normal Nucleoli', 'Mitosis']
X=normalizedData[feature_cols]
y=normalizedData['Class']
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.3, random_state = 42)
dummy_classifier = DummyClassifier(strategy='most_frequent')
dummy_classifier.fit(xTrain,yTrain)
baseline_acc = dummy_classifier.score(xTest,yTest)
print("Baseline Accuracy = ", baseline_acc)


Baseline Accuracy =  0.680952380952381


### 3.  Bagging : Build a generic Bagging ensemble and print the accuracy (4 marks)
---


Hyperparameters:

Base estimator = DecisionTreeClassifier

n_estimators = 10

random_state = 42

---


In [ ]:
# Generic Bagging model
# Enter your code here
model_bagging = BaggingClassifier(base_estimator = DecisionTreeClassifier(), n_estimators = 10, random_state = 42)
model_bagging.fit(xTrain, yTrain)
pred_bagging = model_bagging.predict(xTest)
acc_bagging = accuracy_score(yTest, pred_bagging)

print(' Accuracy = ', acc_bagging)


 Accuracy =  0.9571428571428572


### 4. RandomForest : (7 marks)
#### a) Build a Random Forest model and print the accuracy (4 marks)
---

Constructor arguments: 


n_estimators = 100, max_features = 7 and random_state = 42 


---




In [ ]:
# Random Forest model
# Enter your code here
model_rf = RandomForestClassifier(n_estimators=100, max_features=7, random_state=42)
model_rf.fit(xTrain, yTrain)
predict_rf = model_rf.predict(xTest)
acc_rf = accuracy_score(yTest,predict_rf)
print(acc_rf)


0.9619047619047619


####  b) Calculate the top 3 important features for the above **RandomForest** model and print them (3 marks)

In [ ]:
# Top 3 features for RandomForest
# Enter your code here
feature_importances = model_rf.feature_importances_
features = xTrain.columns
df = pd.DataFrame({'features': features, 'importance': feature_importances}).nlargest(3,'importance')
print(df)



                   features  importance
9                     Class    0.766019
1   Uniformity of Cell Size    0.111952
2  Uniformity of Cell Shape    0.067295


### 5. Boosting: (7 marks)
#### a) Build an AdaBoost model with training data and print the accuracy (4 marks)
---

Hyperparameters:

Base estimator = DecisionTreeClassifier, max_depth = 4

n_estimators = 200

random_state = 42

learning_rate = 0.05


---









In [ ]:
# AdaBoost Classification
# Enter your code here
#base_est = DecisionTreeClassifier (max_depth =4)
ada_boost1 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth =4), n_estimators=200, random_state=42, learning_rate=.05)
ada_boost1.fit(xTrain, yTrain)
predict_ada = ada_boost1.predict(xTest)
acc_ada = accuracy_score(yTest, predict_ada)
#res1 = round(recall_score(yTest, ada_boost1.predict(xTest)),4)
print(acc_ada)



0.9619047619047619


#### b) Calculate the top 3 important features for the above **AdaBoost** model and print them (3 marks)

In [ ]:
# Top 3 features for AdaBoost
# Enter your code here
ada_feature = ada_boost1.feature_importances_
ada_features = xTrain.columns
df2 = pd.DataFrame({'features': ada_features, 'importance': ada_feature}).nlargest(3,'importance')

print(df2)


                  features  importance
9                    Class         1.0
0          Clump Thickness         0.0
1  Uniformity of Cell Size         0.0


### 6. Voting : Using a voting classifier, build an ensemble of RandomForestClassifier, DecisionTreeClassifier, Support Vector Machine and Logistic Regression. (7 marks)


---


Use max_depth = 4, n_estimators = 200, voting = soft

In [ ]:
# Voting Ensemble for Classification
# Enter your code here
rfClf = RandomForestClassifier(n_estimators=200, random_state=0)  
svmClf = SVC(probability=True, random_state=0) 
logClf = LogisticRegression(random_state=0)
clf2 = VotingClassifier(estimators = [('rf',rfClf), ('svm',svmClf), ('log', logClf)], voting='soft') 
clf2.fit(xTrain, yTrain)

clf2_pred = clf2.predict(xTest)
print('Accuracy score', accuracy_score(yTest, clf2_pred))


Accuracy score 0.9666666666666667


### 7. Mention the best model among the above 4 models and its accuracy (1 mark)

In [ ]:
# Write your answer here

#Of all the models, the voting classifier had the best accuracy score at 0.96666666666667
